## Before you start 
Let's make sure that we have access to GPU

In [11]:
!nvidia-smi

Wed Jun  4 09:36:34 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P0             26W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [12]:
import os
HOME = os.getcwd()
print(HOME)

/kaggle/working


## Install dependencies

In [13]:
!pip install -q ultralytics roboflow

## Imports

In [14]:
from roboflow import Roboflow
from google.colab import userdata
from IPython.display import Image
from kaggle_secrets import UserSecretsClient
from IPython.display import Image

## Pull dataset

In [15]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

# ROBOFLOW_API_KEY = userdata.get('ROBOFLOW_API_KEY')
# rf = Roboflow(api_key=ROBOFLOW_API_KEY)

user_secrets = UserSecretsClient()
ROBOFLOW_API_KEY = user_secrets.get_secret("ROBOFLOW_API_KEY")

rf = Roboflow(api_key=ROBOFLOW_API_KEY)

project = rf.workspace("roboflow-jvuqo").project("football-ball-detection-rejhg")
version = project.version(4)
dataset = version.download("yolov8")


mkdir: cannot create directory ‘/kaggle/working/datasets’: File exists
/kaggle/working/datasets
loading Roboflow workspace...
loading Roboflow project...


**NOTE:** Update `data.yaml` structure to align with the latest requirements of the `ultralytics` package.



## Custom file .yaml

In [16]:
import yaml
import os

# dataset.location được định nghĩa từ cell trước, ví dụ: /content/datasets/football-players-detection-14
data_yaml_path = f"{dataset.location}/data.yaml"

print(f"Đường dẫn đến file data.yaml: {data_yaml_path}")
print(f"Thư mục dataset.location: {dataset.location}")

try:
    with open(data_yaml_path, 'r') as f:
        data_config = yaml.safe_load(f)
    if data_config is None:
        data_config = {}
    print("Nội dung data.yaml gốc:", data_config)
except Exception as e:
    print(f"Lỗi khi đọc file {data_yaml_path}: {e}")
    data_config = {}

data_config['train'] = os.path.join(dataset.location, 'train', 'images')
data_config['val'] = os.path.join(dataset.location, 'valid', 'images')

data_config['path'] = dataset.location

try:
    with open(data_yaml_path, 'w') as f:
        yaml.dump(data_config, f, sort_keys=False)
    print(f"Đã cập nhật {data_yaml_path} với đường dẫn tuyệt đối.")
except Exception as e:
    print(f"Lỗi khi ghi file {data_yaml_path}: {e}")

print(f"\nNội dung file {data_yaml_path} SAU KHI CẬP NHẬT:")
!cat {data_yaml_path}
print("---")

Đường dẫn đến file data.yaml: /kaggle/working/datasets/football-ball-detection-4/data.yaml
Thư mục dataset.location: /kaggle/working/datasets/football-ball-detection-4
Nội dung data.yaml gốc: {'names': ['ball'], 'nc': 1, 'roboflow': {'license': 'CC BY 4.0', 'project': 'football-ball-detection-rejhg', 'url': 'https://universe.roboflow.com/roboflow-jvuqo/football-ball-detection-rejhg/dataset/4', 'version': 4, 'workspace': 'roboflow-jvuqo'}, 'test': '../test/images', 'train': '/kaggle/working/datasets/football-ball-detection-4/train/images', 'val': '/kaggle/working/datasets/football-ball-detection-4/valid/images', 'path': '/kaggle/working/datasets/football-ball-detection-4'}
Đã cập nhật /kaggle/working/datasets/football-ball-detection-4/data.yaml với đường dẫn tuyệt đối.

Nội dung file /kaggle/working/datasets/football-ball-detection-4/data.yaml SAU KHI CẬP NHẬT:
names:
- ball
nc: 1
roboflow:
  license: CC BY 4.0
  project: football-ball-detection-rejhg
  url: https://universe.roboflow.co

## Custom training

In [ ]:
%cd {HOME}

!yolo task=detect mode=train model=yolov8x.pt data={dataset.location}/data.yaml batch=4 epochs=50 imgsz=1280 plots=True

/kaggle/working
Ultralytics 8.3.149 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/datasets/football-ball-detection-4/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1280, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8x.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, pat

## Validate custom model

In [ ]:
!ls {HOME}/runs/detect/train/

In [ ]:
!zip -r {HOME}/results.zip {HOME}/runs/detect/train

In [ ]:
%cd {HOME}

!yolo task=detect mode=val model={HOME}/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml imgsz=1280